In [1]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, data_handling, timeframes
import numpy as np
from datetime import datetime, timedelta



In [ ]:
L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list = data_handling.load_collections("./half_month_collections.pkl")

In [5]:
markets_name_dict = {market.name: market for market in M}
market_names = list(markets_name_dict.keys())

In [14]:
def find_previous_weekday(d, weekday):
    days_behind = d.weekday() - weekday
    if days_behind < 0: # Target day is in the previous week
        days_behind += 7
    return d - timedelta(days=days_behind)

# Function to calculate the deadlines for a given input timestamp
def calculate_deadlines_for_input(input_timestamp):
    deadlines = {}

    # RKOM uke: Friday the week before if on a weekday, otherwise Thursday the same week
    if input_timestamp.weekday() < 5:  # Weekday
        deadlines['RKOM uke Weekday'] = find_previous_weekday(input_timestamp - timedelta(days=7), 4)  # Previous Friday
    else:  # Weekend
        deadlines['RKOM uke Weekend'] = find_previous_weekday(input_timestamp, 3)  # Current Thursday

    # FCR D-2: Two days before the input time at 17:30
    deadlines['FCR D-2'] = datetime.combine(input_timestamp.date() - timedelta(days=2), datetime.strptime('17:30', '%H:%M').time())

    # aFRR CM: One day before the input time at 07:30
    deadlines['aFRR'] = datetime.combine(input_timestamp.date() - timedelta(days=1), datetime.strptime('07:30', '%H:%M').time())

    # FCR D-1: One day before the input time at 18:00
    deadlines['FCR D-1'] = datetime.combine(input_timestamp.date() - timedelta(days=1), datetime.strptime('18:00', '%H:%M').time())

    # RK: 45 minutes before the input time
    deadlines['RK'] = input_timestamp - timedelta(minutes=45)

    return deadlines

In [24]:
weekly_timeframe = utils.get_timestampstimeframes.one_week

In [22]:
def make_bids_for_one_week(weekly_timeframe, L,M,H, freq_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m):
    
    for h, hour in enumerate(list(weekly_timeframe)): #19-26.06.2023
        deadlines = calculate_deadlines_for_input(hour)
        print(deadlines)
        
        
    
    return None

In [23]:
make_bids_for_one_week(weekly_timeframe, L,M,H, freq_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m)

TypeError: 'TimeFrame' object is not iterable